In [1]:
import geopandas as gpd
import shapely as sh
import os
import pyrosm
import pandas as pd
import json

In [2]:
os.chdir('/Users/benpage/Documents/GitHub/r5food/data/')

In [3]:
def get_extract(data_dir, location):
    osm_data = pyrosm.get_data(location, directory=data_dir)
    print("Data was downloaded to:", osm_data)
    return osm_data

In [4]:
def get_supermarkets(extract_file, boundry):
    
    boundry_geom = boundry['geometry'].values[0]
    print('Loading extract')
    
    tor_osm = pyrosm.OSM(extract_file, boundry_geom)
    print('Getting supermarkets')
    supermarkets = tor_osm.get_data_by_custom_criteria(custom_filter={'shop':['supermarket']}, 
                                               extra_attributes=['brand'], 
                                               keep_relations=False)
    supermarkets['geometry'] = supermarkets['geometry'].to_crs('EPSG:26917').centroid
    return supermarkets

In [26]:
if __name__ == '__main__':
    os.chdir('/Users/benpage/Documents/GitHub/r5food/data/')
    extract_name = 'Toronto.osm.pbf'
    location = 'Toronto'
    data_dir = '/Users/benpage/Documents/GitHub/r5food/data/'

    tor_boundry = gpd.read_file('tor_boundry.geojson')
    supermarkets = get_supermarkets(extract_name, tor_boundry)
    # market_filter = supermarkets[['id', 'tags', 'addr:city', 'addr:housenumber', 'addr:postcode', 
    #      'addr:place', 'addr:street', 'operator', 'phone', 'ref', 'website',
    #      'brand', 'geometry', 'osm_type', 'alcohol']].sort_values(by='brand')
    # market_filter.to_file('supermarkets.geoJSON')

    supermarkets['brand'] = supermarkets['brand'].fillna(supermarkets['name'])
    replacements = {'Whole Foods Market': 'Whole Foods'}
    supermarkets['brand'] = supermarkets['brand'].replace(replacements)
    supermarkets = supermarkets.dropna(subset=['brand'])
    brand_count = supermarkets['brand'].value_counts()
    brand_count.to_csv('all_brands.csv')
    
    to_remove = brand_count < 2
    to_remove['Bulk Barn'] = True
    
    remove_counts = brand_count.loc[to_remove]
    remove_counts.to_csv('removed_supermarkets_counts.csv')
    
    remove_lst = to_remove[to_remove].index.tolist()
    supermarkets = supermarkets.query("brand not in @remove_lst")
    
    addr_cols = ['addr:housenumber',
                 'addr:street',
                 'addr:city',
                 'addr:postcode']
    
    supermarkets[addr_cols] = supermarkets[addr_cols].fillna('')
    
    supermarkets['address'] = supermarkets[addr_cols].agg(''.join, axis=1).replace('', None)

    supermarkets['lat'] = supermarkets.geometry.x
    supermarkets['lon'] = supermarkets.geometry.x
    supermarkets = supermarkets.reset_index(drop=True)
    
    supermarkets['id_2'] = supermarkets.index + 1

    supermarkets[['id', 'id_2']].to_csv('supermarkets_id.csv')

    col_keep = ['id_2', 'address','brand', 'lat', 'lon', 'geometry']
    supermarkets_clean = supermarkets[col_keep].copy()

    supermarkets_clean = supermarkets_clean.rename(columns = {'id_2':'id'})
    supermarkets_clean.to_file('supermarkets.geoJSON')
    
    supermarkets_clean = supermarkets[col_keep]
    
    # supermarkets_clean

Loading extract
Getting supermarkets
